### Use this markdown to generate the predictors


In [ ]:
#imported libraries
import pandas as pd
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor

#Creating of the input data
downloads = pd.read_csv('train_app_downloads.csv')
reviews = pd.read_csv('train_app_review.csv')
ratings = pd.read_csv('train_app_rating.csv')
usages = pd.read_csv('train_usage.csv')
revenues = pd.read_csv('train_revenue.csv')
output = pd.read_csv('train_final_downloads.csv')
dateRange = pd.date_range('2015-03-01', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))

sentiment = pd.read_csv('sentiment.csv',header=-1).ix[:,0]
sentiment.values[sentiment.values==0.0] = 0.5
reviews["sentiment_score"] = sentiment.values 

#We map -1 to 0 in the downloads (there are no 0 in the initial data)
replacementValue=0
downloads = downloads.replace(-1,replacementValue)

#Minor corrections
ratings = ratings.rename(columns={'start1': 'star1'})
ratings = pd.merge(downloads.drop(dateRange,1), ratings.drop('Unnamed: 0', 1), how='left',
                   on=["id","name","category"]).replace("NaN",replacementValue)

In [ ]:
output = pd.read_csv('train_final_downloads.csv')

In [ ]:
#rank = [0]* output.shape[0]
top = [0]* output.shape[0]
#dic = {}
#ranklist = list(output.ix[:,"cumulative_downloads_2016-02"].argsort()[::-1])

#for i in range(len(ranklist)):
 #   dic[ranklist[i]] = i
#for i in range(output.shape[0]):
 #   rank[i] = dic[i]
#output["rank"] = rank


In [ ]:
top_index = output.sort_values(by = "cumulative_downloads_2016-02" , ascending = False).iloc[0:6000,:].index #drop apps with the lowest final downloads
top_index
for i in range(output.shape[0]):
    if i in top_index: top[i] = 1
output["top"] = top



In [ ]:
output.head()

In [ ]:
output = output.sort_values(by = "cumulative_downloads_2016-02" , ascending = False)

#### Initialize the predictors matrix

In [ ]:
predictors = pd.concat([downloads["id"],downloads["name"],downloads["category"],downloads["device"]],
                       axis=1,keys=["id","name","category","device"])
predictors.head()

#### Use the train_app_downloads.csv file

In [ ]:
#Generate the logWeekly average
def generator_weekAvg(inp,w):
    if (np.count_nonzero(inp[5+w*7:12+w*7] - replacementValue*np.ones(len(inp[5+w*7:12+w*7]))) == 0):
        return 0
    return  math.log(1.0*sum(inp[5+w*7:12+w*7])/np.count_nonzero(inp[5+w*7:12+w*7] 
                                                                 - replacementValue*np.ones(len(inp[5+w*7:12+w*7]))))

for w in range(8):
    predictors["week_"+str(w+1)] = downloads.apply(generator_weekAvg,axis=1,args=(w,))

In [ ]:
#Generate the daily average
def generator_dailyAvg(inp):
    if (np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))) == 0):
        return 0
    return  (1.0*sum(inp[5:])/np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))))
    #return  math.log(1.0*sum(inp[5:])/len(inp[5:]))

predictors["daily_avg"] = downloads.apply(generator_dailyAvg,axis=1)
#This one is bad

In [ ]:
#Generate the polynomial coefficients
def generator_coef(inp, coef):
    return  np.polyfit(range(56),inp[5:],coef)[0]
#Redo by ignoring the -1 / 0 ? w = [1110011]     w = np.not_equal(inp[5:],np.ones(len(inp[5:]))*replacementValue

for c in range(4):
    predictors["coef_"+str(c)] = downloads.apply(generator_coef,axis=1,args=(c,))

In [ ]:
#Generate the step max and min (we ignore the values of 0)
def generator_maxStep(inp,maximum):
    if (np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))) == 0):
        return 0
    m = 0
    for d in range(1,56):
        if (inp[5+d]!=replacementValue and inp[4+d]!=replacementValue):
            c = (inp[5+d]-inp[4+d])
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    return m
                
predictors["maxStep"] = downloads.apply(generator_maxStep,axis=1,args=(True,))
predictors["minStep"] = downloads.apply(generator_maxStep,axis=1,args=(False,))

In [ ]:
#Standard Deviation
def generator_std(inp):
    if (np.count_nonzero(inp[5:] - replacementValue*np.ones(len(inp[5:]))) == 0):
        return 0
    return np.std(inp[5:])

predictors["std"] = downloads.apply(generator_std,axis = 1)

In [ ]:
#Number of missing Values
def generator_missing(inp):
    return list(inp[5:]).count(replacementValue)
    #return np.count_zero(inp[5:] -replacementValue*np.ones(len(inp[5:])))

predictors["nb_missing"] = downloads.apply(generator_missing,axis = 1)

In [ ]:
#device
def generator_iphone(inp):
    if (inp[4] == "iphone"):
        return 1
    else:
        return 0

def generator_ipad(inp):
    if(inp[4] == "ipad"):
        return 1
    else:
        return 0
    
predictors["iphone"] = downloads.apply(generator_iphone,axis = 1)
predictors["ipad"] = downloads.apply(generator_ipad,axis = 1)

In [ ]:
#Categories
def generator_categories(inp,cat):
    if (inp[3] == cat):
        return 1
    else:
        return 0
    

for cat in list(set(downloads["category"])):
    predictors[cat] = downloads.apply(generator_categories,axis = 1,args=(cat,))

#### Use the train_app_rating.csv file

In [ ]:
#Generate ratings   This is bad
def generator_weightSum(inp):
    return -2*inp[5]-inp[6]+inp[8]+2*inp[9]

predictors["ratingsWeightSum"] = ratings.apply(generator_weightSum,axis=1)

In [ ]:
avg_reviews = reviews.groupby('id').agg('mean')
avg_reviews['rating']
predictors['avg_review'] = predictors.join(avg_reviews['rating'],on='id')['rating'].replace("NaN",0)

In [ ]:
avg_reviews = reviews.groupby('id').agg('var')
avg_reviews['rating']
predictors['var_review'] = predictors.join(avg_reviews['rating'],on='id')['rating'].replace("NaN",0)

In [ ]:
#raw ratings
predictors = pd.merge(predictors, ratings.drop('Unnamed: 0', 1), how='left',
                   on=["id","name","category","device"]).replace("NaN",replacementValue)

#### Use the sentiment score

In [ ]:
#Later we can compute the weighted average of sentiment scores based on reviewers.
#add positive and negative columns to indicate the app's popularity
avg_score = [0]*predictors.shape[0]
predictors["positive"] = [0]*predictors.shape[0]
predictors["negative"] = [0]*predictors.shape[0]
for i in range(predictors.shape[0]):
    avg_score[i] = reviews.ix[reviews["id"]==predictors.id[i],:]["sentiment_score"].mean()
    if avg_score[i]>0.52: 
        predictors["positive"].values[i] = 1
    elif avg_score[i]<0.48: predictors["negative"].values[i] = 1
predictors["avg_sentiment_score"] = avg_score

 

#### Use coeficients of metrics 

In [ ]:
for i in range(4):
    predictors["m"+str(i+1)+"_max"] = np.zeros(predictors.shape[0]) 
    predictors["m"+str(i+1)+"_min"] = np.zeros(predictors.shape[0])
    predictors["m"+str(i+1)+"_mean"] = np.zeros(predictors.shape[0])
    predictors["m"+str(i+1)+"_std"] = np.zeros(predictors.shape[0])
    for j in range(3):
        predictors["m"+str(i+1)+"_coef_"+str(j)] = np.zeros(predictors.shape[0])

for i in range(predictors.shape[0]):
    if predictors["id"].values[i] not in usages["id"].values: continue
    for j in range(4):
        tmp = usages.ix[usages["id"] == predictors["id"].values[i],:]
        time_series = np.array(tmp.ix[tmp["metric"] == j+1,6:14])[0]   
        if -1 in time_series: continue
        predictors["m"+str(j+1)+"_max"].values[i] = time_series.max()
        predictors["m"+str(j+1)+"_min"].values[i] = time_series.min()
        predictors["m"+str(j+1)+"_mean"].values[i] = time_series.mean()
        predictors["m"+str(j+1)+"_std"].values[i] = time_series.std()
        X = np.array(range(8))
        fit = np.polyfit(X,time_series,2)
        for k in range(3):
            predictors["m"+str(j+1)+"_coef_"+str(k)].values[i] = fit[2-k]


#### Use coeficients of revenue

In [ ]:
for j in range(3):  
    predictors["rev_coef_"+str(j)] = np.zeros(predictors.shape[0])
    predictors["rev_max"] = np.zeros(predictors.shape[0])
    predictors["rev_min"] = np.zeros(predictors.shape[0])
    predictors["rev_mean"] = np.zeros(predictors.shape[0])
    predictors["rev_std"] = np.zeros(predictors.shape[0])
for i in range(predictors.shape[0]):
    if predictors["id"].values[i] in revenues["id"].values: 
        curr_rev  = revenues.ix[revenues["id"]== predictors["id"].values[i],:]
        if predictors["device"].values[i] in curr_rev["device"].values:
            curr_rev = curr_rev.ix[curr_rev["device"] == predictors["device"].values[i],:]
            time_series = np.array(curr_rev.ix[:,5:61])[0]
            Y = []
            for t in range(49): Y.append(time_series[t:t+7].sum()) #reduce seasonality
            Y = np.array(Y)
            if -1 in time_series: continue 
            predictors["rev_max"].values[i] = Y.max()
            predictors["rev_min"].values[i] = Y.min()
            predictors["rev_mean"].values[i] = Y.mean()
            predictors["rev_std"].values[i] = Y.std()
            X = np.array(range(49))
            fit = np.polyfit(X,Y,2)
            for k in range(3):  predictors["rev_coef_"+str(k)].values[i] = fit[2-k]            

In [ ]:
print predictors.columns

## How good did the predictor perform

In [ ]:
#This is the metric we use to determine our performance
def metric(y_pred,y_test, top  = 20):
    return len(set([i[0] for i in sorted(enumerate(y_pred), key=lambda x:x[1],reverse=True)][0:top])
               .intersection([i[0] for i in sorted(enumerate(y_test), key=lambda x:x[1],reverse=True)][0:top]))

## To csv



In [ ]:
predictors.to_csv("predictors.csv")

In [ ]:
predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)

In [ ]:
predictors.head()

In [ ]:
print predictors.shape
print output.shape

In [ ]:
#predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
#predictors = predictors.drop('Unnamed: 0', 1)
predictors = predictors.fillna(0)

#Standardize
#for p in [ 'daily_avg', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'ratingsWeightSum', 'star1', 'star2', 'star3', 'star4', 'star5']:
#    predictors[p] = (predictors[p] - predictors[p].mean())/predictors[p].std() 

#drop daily
#predictors = predictors.drop("week1",1)
#predictors = predictors.drop("week2",1)
#predictors = predictors.drop("week3",1)
#predictors = predictors.drop("week4",1)
#predictors = predictors.drop("week5",1)
#predictors = predictors.drop("week6",1)
#predictors = predictors.drop("week7",1)
#predictors = predictors.drop("week8",1)
# predictors = predictors.drop("daily_avg",1)
# predictors = predictors.drop("star1",1)
# predictors = predictors.drop("star2",1)
# predictors = predictors.drop("star3",1)
# predictors = predictors.drop("star4",1)
# predictors = predictors.drop("star5",1)
# predictors = predictors.drop(list(set(downloads["category"])),1)
# predictors = predictors.drop("ratingsWeightSum",1)
#predictors = predictors.drop("coef_0",1)
# predictors = predictors.drop("coef_1",1)
# predictors = predictors.drop("coef_2",1)
# predictors = predictors.drop("coef_3",1)
# predictors = predictors.drop("maxStep",1)
# predictors = predictors.drop("minStep",1)
# predictors = predictors.drop("std",1)
# predictors = predictors.drop("nb_missing",1)
#predictors = predictors.drop("iphone",1)
# predictors = predictors.drop("ipad",1)

# predictors = predictors.drop('avg_sentiment_score',1)
# predictors = predictors.drop('positive',1)
# predictors = predictors.drop('negative',1)
#predictors = predictors.drop('m1_coef_0',1)
# predictors = predictors.drop('m1_coef_1',1)
# predictors = predictors.drop('m1_coef_2',1)
#predictors = predictors.drop('m2_coef_0',1)
#predictors = predictors.drop('m2_coef_1',1)
# predictors = predictors.drop('m2_coef_2',1)
#predictors = predictors.drop('m3_coef_0',1)
# predictors = predictors.drop('m3_coef_1',1)
# predictors = predictors.drop('m3_coef_2',1)
#predictors = predictors.drop('m4_coef_0',1)
# predictors = predictors.drop('m4_coef_1',1)
# predictors = predictors.drop('m4_coef_2',1)
#predictors = predictors.drop('rev_coef_0',1)
#predictors = predictors.drop('rev_coef_1',1)
#predictors = predictors.drop('rev_coef_2',1)






test_frac = 0.31  #Fraction of test points
N = 20   #number of iterations
l=100  #Size of the top
y_pred = np.array([0]*predictors.shape[0])   


np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:], output.as_matrix()[:,7], test_size=test_frac, random_state=r)  
    #print list(y_train)
    mod= RandomForestClassifier(max_features = 1.0/3.0,n_estimators = 100) 
    mod.fit(X_train,list(y_train))    
    y_pred = y_pred + np.array(mod.predict(predictors.as_matrix()[:,4:]))

top_pred = []
for i in range(predictors.shape[0]):
    if y_pred[i] >N/2: top_pred.append(i)

    


In [ ]:
#predictors = pd.read_csv('predictors.csv').drop('Unnamed: 0', 1)
#predictors = predictors.drop('Unnamed: 0', 1)
#predictors = predictors.ix[top_pred,:]
#output = output.ix[top_pred,:]
#Standardize
#for p in [ 'daily_avg', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'ratingsWeightSum', 'star1', 'star2', 'star3', 'star4', 'star5']:
#    predictors[p] = (predictors[p] - predictors[p].mean())/predictors[p].std() 

#drop daily
#predictors = predictors.drop("week1",1)
#predictors = predictors.drop("week2",1)
#predictors = predictors.drop("week3",1)
#predictors = predictors.drop("week4",1)
#predictors = predictors.drop("week5",1)
#predictors = predictors.drop("week6",1)
#predictors = predictors.drop("week7",1)
#predictors = predictors.drop("week8",1)
# predictors = predictors.drop("daily_avg",1)
# predictors = predictors.drop("star1",1)
# predictors = predictors.drop("star2",1)
# predictors = predictors.drop("star3",1)
# predictors = predictors.drop("star4",1)
# predictors = predictors.drop("star5",1)
# predictors = predictors.drop(list(set(downloads["category"])),1)
# predictors = predictors.drop("ratingsWeightSum",1)
#predictors = predictors.drop("coef_0",1)
# predictors = predictors.drop("coef_1",1)
# predictors = predictors.drop("coef_2",1)
# predictors = predictors.drop("coef_3",1)
# predictors = predictors.drop("maxStep",1)
# predictors = predictors.drop("minStep",1)
# predictors = predictors.drop("std",1)
# predictors = predictors.drop("nb_missing",1)
#predictors = predictors.drop("iphone",1)
# predictors = predictors.drop("ipad",1)

# predictors = predictors.drop('avg_sentiment_score',1)
# predictors = predictors.drop('positive',1)
# predictors = predictors.drop('negative',1)
#predictors = predictors.drop('m1_coef_0',1)
# predictors = predictors.drop('m1_coef_1',1)
# predictors = predictors.drop('m1_coef_2',1)
#predictors = predictors.drop('m2_coef_0',1)
#predictors = predictors.drop('m2_coef_1',1)
# predictors = predictors.drop('m2_coef_2',1)
#predictors = predictors.drop('m3_coef_0',1)
# predictors = predictors.drop('m3_coef_1',1)
# predictors = predictors.drop('m3_coef_2',1)
#predictors = predictors.drop('m4_coef_0',1)
# predictors = predictors.drop('m4_coef_1',1)
# predictors = predictors.drop('m4_coef_2',1)
#predictors = predictors.drop('rev_coef_0',1)
#predictors = predictors.drop('rev_coef_1',1)
#predictors = predictors.drop('rev_coef_2',1)



NUM = predictors.shape[0]
old_top = []
new_top = []

test_frac = 0.31  #Fraction of test points
N = 20   #number of iterations
l=100  #Size of the top
np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:12], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
    #X_test = X_test[0:NUM/5]
    #y_test = y_test[0:NUM/5]
    old_mod=linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train)
    old_y_pred =  old_mod.predict(X_test)
    old_top.append(metric(old_y_pred,y_test,l))
    
'''np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
    X_test = X_test[0:10000]
    y_test = y_test[0:10000]
    #mod=linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train)
    mod=linear_model.Lasso(fit_intercept=False).fit(X_train,y_train) 
    y_pred =  mod.predict(X_test)
    new_top.append(metric(y_pred,y_test,l))'''

np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
    #X_test = X_test[0:NUM/5]
    #y_test = y_test[0:NUM/5]
    #mod=linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train)
    #mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100) 
    #mod.fit(X_train,y_train)
    #importance_list = list(reversed(np.argsort(mod.feature_importances_)))[:50]
    mod=linear_model.Lasso(fit_intercept=False).fit(X_train,y_train) 
    importance_list =[i for i in  range(len(mod.coef_))if abs(mod.coef_[i])>10]
    
    '''X_boost_train = X_train[:,importance_list]
    y_boost_train = y_train
    X_boost_test = X_test[:,importance_list]
    y_boost_test = y_test
    params = {'n_estimators': 200, 'max_depth': 1, 'learning_rate': 0.01, 'loss': 'ls'}
    boost = GradientBoostingRegressor(**params) 
    boost.fit(X_boost_train,y_boost_train)
    y_pred =  boost.predict(X_boost_test)'''
    
    X_rf_train = X_train[:,importance_list]
    y_rf_train = y_train
    X_rf_test = X_test[:,importance_list]
    y_rf_test = y_test
    mod2= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 200) 
    mod2.fit(X_rf_train,y_rf_train)
    
    
    
    
    y_pred = mod2.predict(X_rf_test)
    new_top.append(metric(y_pred,y_rf_test,l))
    
if (N<300):
    fig = plt.figure()
    plt.plot(range(1,N),old_top,label="old")
    plt.plot(range(1,N),new_top,label="performance")
    plt.legend(loc='best')
    plt.show()

print 1.0*sum(old_top)/len(old_top)
print list(predictors.columns.values)
print 1.0*sum(new_top)/len(new_top)

In [ ]:
NUM

In [ ]:
np.random.seed(1)
for i in range(1,N):
    r = np.random.randint(1,429496729)
    X_train, X_test, y_train, y_test = train_test_split(predictors.as_matrix()[:,4:], output.as_matrix()[:,5], test_size=test_frac, random_state=r)
    X_test = X_test[0:10000]
    y_test = y_test[0:10000]
    #mod=linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train)
    mod= RandomForestRegressor(max_features = 1.0/3.0,n_estimators = 100) 
    mod.fit(X_train,y_train)
    '''importance_list = list(reversed(np.argsort(mod.feature_importances_)))[:50]
    X_boost_train = X_train[:,importance_list]
    y_boost_train = y_train
    X_boost_test = X_test[:,importance_list]
    y_boost_test = y_test
    params = {'n_estimators': 200, 'max_depth': 1, 'learning_rate': 0.01, 'loss': 'ls'}
    boost = GradientBoostingRegressor(**params) 
    boost.fit(X_boost_train,y_boost_train)
    y_pred =  boost.predict(X_boost_test)'''
    y_pred = mod.predict(X_test)
    new_top.append(metric(y_pred,y_test,l))


['id', 'name', 'category', 'device', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'daily_avg', 'coef_0', 'coef_1', 'coef_2', 'coef_3', 'maxStep', 'minStep', 'std', 'nb_missing', 'iphone', 'ipad', 'Productivity', 'Entertainment', 'Travel', 'Sports', 'Music', 'Shopping', 'Finance', 'Business', 'Navigation', 'Food and Drink', 'Utilities', 'Newsstand', 'Health and Fitness', 'News', 'Lifestyle', 'Medical', 'Weather', 'Games', 'Catalogs', 'Social Networking', 'Photo and Video', 'Reference', 'Books', 'Education', 'ratingsWeightSum', 'avg_review', 'var_review', 'star1', 'star2', 'star3', 'star4', 'star5', 'positive', 'negative', 'avg_sentiment_score', 'm1_max', 'm1_min', 'm1_mean', 'm1_std', 'm1_coef_1', 'm1_coef_2', 'm2_max', 'm2_min', 'm2_mean', 'm2_std', 'm2_coef_1', 'm2_coef_2', 'm3_max', 'm3_min', 'm3_mean', 'm3_std', 'm3_coef_1', 'm3_coef_2', 'm4_max', 'm4_min', 'm4_mean', 'm4_std', 'm4_coef_1', 'm4_coef_2', 'rev_coef_1', 'rev_coef_2', 'rev_max', 'rev_min', 'rev_mean', 'rev_std']
58.0

54.8947368421  
55.0526315789  #with #missing and Lasso <br/>
54.7894736842  #including raw ratings Lasso :( <br/>
55.0526315789  #with #missing weightedSumRatings and Lasso <br/>
55.1052631579  #adding the categories <br/>
55.3684210526  #adding average sentiment score and positive/negative label <br/>
55.5263157895  #adding coeficients of usages


With 10000 only:
54.8947368421 all

In [ ]:
avg_reviews = reviews.groupby('id').agg('mean')
avg_reviews['rating']
predictors['avg_review'] = predictors.join(avg_reviews['rating'],on='id')['rating'].replace("NaN",0)

In [ ]:
avg_reviews = reviews.groupby('id').agg('var')
avg_reviews['rating']
predictors['var_review'] = predictors.join(avg_reviews['rating'],on='id')['rating'].replace("NaN",0)

In [ ]:
predictors

In [ ]:
predictors = predictors.drop(['avg_sentiment_score'],1)

In [ ]:
predictors


In [ ]:
predictors.mean()

In [ ]:
predictors["avg_sentiment_score"][16].replace('nan',0)